In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" 
#Change the path of some HF environment variables to store the download data (model and dataset) from the hub to a choosen location
os.environ['HF_HOME'] = "../.cache"
os.environ['HF_HUB_CACHE'] = "../.cache"
os.environ['HF_DATASETS_CACHE'] = "../.cache"
from transformers import PaliGemmaProcessor, AutoTokenizer, DonutImageProcessor
from modeling_divedoc import DIVEdoc
from processing_divedoc import PaliGemmaProcessor, get_processor
from datasets import load_dataset

/home/rbencharef/miniconda3/envs/divedoc-plateform-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_token = os.getenv("HF_TOKEN")

In [3]:
if hf_token:
    print("cc")

cc


In [3]:
model = DIVEdoc.from_pretrained(
    "JayRay5/DIVE-Doc-FRD",
    trust_remote_code=True)

Vision config in end-to-end model: swinpam
[64, 64]
[64, 64]


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 41.01it/s]


In [ ]:
assert all(not p.requires_grad for p in model.parameters())

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-ft-docvqa-896",token=hf_token)
image_processor = DonutImageProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
image_processor.image_seq_length = 4096
image_processor.size["height"],image_processor.size["width"] = model.config.vision_config.encoder_config.image_size[0],model.config.vision_config.encoder_config.image_size[1]

In [5]:
#processor = PaliGemmaProcessor(tokenizer=tokenizer, image_processor=image_processor)
processor = get_processor(hf_token,2048,2048,4096)

In [6]:
test_dataset = load_dataset("pixparse/docvqa-single-page-questions", split="test",streaming=True)
batch_dataset = test_dataset.batch(2) 
iterative_dataset = iter(batch_dataset)
batch = next(iterative_dataset)

KeyboardInterrupt: 

In [40]:
#preprocessing 
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
txt = batch["question"]
model_inputs = processor(text=txt, images=imgs, return_tensors="pt",padding=True)
input_len = model_inputs["input_ids"].shape[-1]
    

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


In [44]:
original_text = "DIVE-Doc is a 2.5B end-to-end VLM!"
        
encoded = processor.tokenizer(original_text, return_tensors="pt",padding=True)
input_ids = encoded["input_ids"][0] 

decoded_text = processor.decode(input_ids, skip_special_tokens=True)

print(decoded_text)

DIVE-Doc is a 2.5B end-to-end VLM!


In [11]:
model_inputs["attention_mask"].shape

torch.Size([2, 4103])

In [26]:
import torch
assert isinstance(model_inputs["input_ids"],torch.Tensor)

In [25]:
from transformers import BatchEncoding,BatchFeature
assert isinstance(model_inputs,(dict, BatchFeature))

In [14]:
processor.image_processor.size['height']

2048

In [33]:
import random
random_list = [random.randint(1, 100) for _ in range(10)]
processor.decode(random_list, skip_special_tokens=True)

'<unused62>[@BOS@]<unused4><unused12><unused20><unused20><unused2><unused36><unused68><unused70>'

In [ ]:
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
processor.image_processor(images=imgs[0])

{'pixel_values': [array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]]],
      shape=(3, 2560, 1920), dtype=float32)]}

In [ ]:
model.config

DIVEdocConfig {
  "_attn_implementation_autoset": true,
  "_vocab_size": 257152,
  "architectures": [
    "DIVEdoc"
  ],
  "auto_map": {
    "AutoConfig": "JayRay5/DIVE-Doc-FRD--configuration_divedoc.DIVEdocConfig",
    "AutoModelForCausalLM": "JayRay5/DIVE-Doc-FRD--modeling_divedoc.DIVEdoc"
  },
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "image_token_index": 257152,
  "model_type": "DIVEdoc",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "head_dim": 256,
    "hidden_act": "gelu_pytorch_tanh",
    "hidden_activation": null,
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "intermediate_size": 16384,
    "max_position_embeddings": 8192,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 4096,
    "num_key_value_heads": 1,
    "rms_norm_eps": 1e-06,
    "rope_theta": 10000.0,
    "torch_dtype": "float32",
    "use